# Modelos preditivos de queimadas no Brasil

---

## Introdução

O tema da preservação do meio ambiente está desde sempre presente no nosso dia a dia, e é um tema cada vez mais importante, dada a constante exploração da natureza e suas consequências que ameaçam a sobrevivência da própria espécie humana. 

As queimadas são uns dos principais problemas, que pelo menos no Brasil, (ainda) são consideradas crimes contra a natureza, se não feitas com licença do orgão ambiental competente.

As queimadas devastam áreas inteiras de vegetação, matam e expulsam a fauna, prejudicam o solo, emitem gases poluentes e fumaça que fazem mal à saúde dos animais (humanos inclusos), além de aumentar a temperatura como um todo, contribuindo para o famigerado aquecimento global e efeito estufa.

Existem dois fatores principais para as queimadas: o natural e o humano. No primeiro, se deve a incidência de descargas elétricas em áreas de pastagens, e também atividades vulcânicas. No segundo caso, são provocadas pelo ação humana, deliberada ou não, como queimadas para limpeza, balões, bitucas de cigarro, entre outras causas.

Para quem pratica a atividade agropecuária, é comum que nas épocas de maior seca, seja aproveitado para executar as tais limpezas no solo para o plantio. No caso de países com uma economia desindustrializada e dependente da exportação de commodities agrícolas, há um aumento constante dessas atividades, e com isso uma expansão cada vez maior das fronteiras agrícolas. Para que isso possa ocorrer, taca-se fogo em tudo o que há pela frente, de forma criminosa, em hectares e mais hectares de natureza.

Uma das soluções passa nas opções que o país faz, em investir tão pouco numa econômia industrial, e tanto na exportação de produtos vindos da agropecuária. Outra solução _ que não resolve _ é o da exploração responsável, onde nesse caso são adotados sistemas de monitoramento e fiscalicação eficientes por parte do poder público, e leis severas que punem de fato quem faz os chamados desmatamentos ilegais. A questão é o que o país entende por desmatamento ilegal. Achar que apenas agir dentro da lei não é suficiente, no sentido de proteger o meio ambiente, se tais leis continuarem a ser tão permissivas.

Seja como for, há de pelo menos começarmos a fazer o mínimo, e ir elevando a consciência popular para as questões do meio ambiente.

Neste relatório técnico, o objetivo é através de algoritmos, criar previsões de queimadas.

**2 - Abordagem de Aprendizado de Máquina para tratar o problema (descrição da tarefa e algoritmos a serem utilizados)**

Pensar a partir da análise já iniciada abaixo, quais algoritmos serão utilizados. Acredito que análises preditivas são as mais adequadas, para criar um relatório para os usuários que será capaz de apontar quais locais estão mais propensos a terem queimadas, e em qual época, e precisam de mais atenção das autoridades e da sociedade civil.


## Coleta de Dados

Os dados desta análise foram coletados a partir do Portal Queimadas do INPE (Instituto Nacional de Pesquisas Espaciais). Foram feitas extrações do período de 8 anos, ou seja, entre janeiro de 2015 e junho de 2022. A escolha desse período se deu pela completude dos dados no portal, valores de risco fogo, frp, dias sem chuva, por exemplo, não eram coletados previamente a esse período.

Os filtros utilizados foram:

- Continentes: América do Sul
- Países: Brasil
- Estados: Todos os estados
- Data Início: 01/01/ano
- Data Fim: 31/12/ano
- Satélites: Satélite de referência (Aqua Tarde)
- Biomas: Todos
- Formato de exportação: CSV

Satélite:

- Aqua: https://www.embrapa.br/satelites-de-monitoramento/missoes/aqua
- Sensor MODIS: https://mundogeo.com/2004/05/23/o-sensor-modis-a-bordo-das-plataformas-terra-e-acqua/

In [1]:
# Bibliotecas

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from datetime import datetime, timezone, timedelta, time, date
from os.path import exists

%load_ext autoreload
%autoreload 2

from Funcoes import Funcoes

## Processamento/Tratamento de Dados

Como o portal limita a coleta de dados a no máximo 1 ano, foram feitas coletas anuais de dados. Portanto, há a necessidade de unir todos os arquivos csv num único.

Mas esse processo não precisa ser repetido sempre. Na primeira execução do projeto ele verifica se o arquivo concatenado foi criado, se não o cria.

In [2]:
# importar dados

arquivo_csv = 'dados/focos_queimadas.csv'

if not exists(arquivo_csv):
    Funcoes.unir_arquivos()
    
df_focos_queimadas = pd.read_csv(arquivo_csv, sep=',', encoding='latin 1')

In [3]:
df_focos_queimadas.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2015/11/07 16:30:00,AQUA_M-T,Brasil,PARA,PORTEL,Amazonia,4.0,0.1,0.6,-2.245,-50.667,NaN
1,2015/04/02 16:46:00,AQUA_M-T,Brasil,TOCANTINS,PEIXE,Cerrado,1.0,0.1,0.3,-11.994,-48.548,NaN
2,2015/03/23 17:47:00,AQUA_M-T,Brasil,MATO GROSSO,FELIZ NATAL,Amazonia,3.0,0.4,0.2,-12.185,-54.405,NaN
3,2015/02/21 17:39:00,AQUA_M-T,Brasil,RORAIMA,IRACEMA,Amazonia,20.0,0.0,1.0,2.247,-61.527,NaN
4,2015/08/02 17:25:00,AQUA_M-T,Brasil,PARA,ORIXIMINA,Amazonia,0.0,0.1,0.8,-1.754,-55.686,NaN


In [4]:
df_focos_queimadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1368553 entries, 0 to 1368552
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   datahora      1368553 non-null  object 
 1   satelite      1368553 non-null  object 
 2   pais          1368553 non-null  object 
 3   estado        1368553 non-null  object 
 4   municipio     1368553 non-null  object 
 5   bioma         1368553 non-null  object 
 6   diasemchuva   1367223 non-null  float64
 7   precipitacao  1367223 non-null  float64
 8   riscofogo     1367223 non-null  float64
 9   latitude      1368553 non-null  float64
 10  longitude     1368553 non-null  float64
 11  frp           757417 non-null   float64
dtypes: float64(6), object(6)
memory usage: 125.3+ MB


De cara é possível perceber que existem dados faltantes para `diasemchuva`, `precipitacao`, `riscofogo` e principalmente `frp`. Depois será necessário verificar como proceder nestes casos.

In [5]:
df_focos_queimadas.describe()

,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
count,1.367223e+06,1.367223e+06,1.367223e+06,1.368553e+06,1.368553e+06,757417.000000
mean,9.504688e+00,7.087691e-01,-4.706983e+00,-1.031434e+01,-5.207692e+01,62.193874
std,8.999016e+01,2.981452e+00,7.363247e+01,6.385679e+00,7.709857e+00,139.726603
min,-9.990000e+02,0.000000e+00,-9.990000e+02,-3.370900e+01,-7.367366e+01,0.000000
25%,1.000000e+00,0.000000e+00,5.000000e-01,-1.364400e+01,-5.704700e+01,15.000000
50%,5.000000e+00,0.000000e+00,1.000000e+00,-9.390000e+00,-5.101300e+01,28.200000
75%,1.700000e+01,1.000000e-01,1.000000e+00,-6.033000e+00,-4.605900e+01,59.000000
max,8.270000e+02,1.430000e+02,1.000000e+00,5.154000e+00,-3.481600e+01,9612.200000


Das colunas que temos, vamos trabalhar com as seguintes:

- **datahora**: Horário de referência da passagem do satélite segundo o fuso horário de Greenwich (GMT);
- **satelite**: Nome do algoritmo utilizado e referência ao satélite provedor da imagem;
- **pais**: Nome do país;
- **estado**: Nome da unidade federativa;
- **municipio**: Nome do município. Para o Brasil foi utilizado como referência o dado do IBGE 2000;
- **bioma** uma unidade biológica ou espaço geográfico cujas características específicas são definidas pelo macroclima, o solo, a altitude, dentre outros critérios;
- **diasemchuva**: Número de dias sem chuva até a detecção do foco;
- **precipitacao**: Valor da precipitação acumulada no dia até o momento da detecção do foco (milímetros por dia);
- **riscofogo**: Valor do Risco de Fogo previsto para o dia da detecção do foco (Vai de 0 até 1, e o valor 777.7 significa que é valor inválido);
- **latitude**: Latitude do foco do incêndio 
- **longitude**: Longitude do foco do incêndio
- **frp**: Fire Radiative Power (Energia Radiativa do Fogo, intensidade da queimada), valor em MW (megawatts).

### Tratar dados

Excluir colunas indesejadas e procurar por valores nulos e faltantes.

In [6]:
df_focos_queimadas.drop(['satelite', 'pais'], axis=1, inplace=True)
df_focos_queimadas.describe()

,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
count,1.367223e+06,1.367223e+06,1.367223e+06,1.368553e+06,1.368553e+06,757417.000000
mean,9.504688e+00,7.087691e-01,-4.706983e+00,-1.031434e+01,-5.207692e+01,62.193874
std,8.999016e+01,2.981452e+00,7.363247e+01,6.385679e+00,7.709857e+00,139.726603
min,-9.990000e+02,0.000000e+00,-9.990000e+02,-3.370900e+01,-7.367366e+01,0.000000
25%,1.000000e+00,0.000000e+00,5.000000e-01,-1.364400e+01,-5.704700e+01,15.000000
50%,5.000000e+00,0.000000e+00,1.000000e+00,-9.390000e+00,-5.101300e+01,28.200000
75%,1.700000e+01,1.000000e-01,1.000000e+00,-6.033000e+00,-4.605900e+01,59.000000
max,8.270000e+02,1.430000e+02,1.000000e+00,5.154000e+00,-3.481600e+01,9612.200000


Encontrado vazios (NA) nas colunas diasemchuva, precipitacao, riscofogo e frp.

In [7]:
df_focos_queimadas.isna().sum()

datahora             0
estado               0
municipio            0
bioma                0
diasemchuva       1330
precipitacao      1330
riscofogo         1330
latitude             0
longitude            0
frp             611136
dtype: int64

In [8]:
# diasemchuva, precipitacao, riscofogo

(1330 / df_focos_queimadas.size) * 100

0.009718293701449632

No caso das colunas que tem poucos valores vazios (menos de 1%), zerar estes valores não vai causar nenhum prejuízo.

In [9]:
# frp

(611136 / df_focos_queimadas.size) * 100

4.465563262803852

No caso do campo `frp`, os dados vazios são cerca de 4% do dataset, mas numa das explicações sobre a coleta de dados dos satélites, este valor nem sempre consegue ser medido, porque pode ter sido uma queimada de baixa intensidade, então ele será mantidado tal como veio.

In [10]:
# Zerando os valores riscofogo, diasemchuva, precipitacao quando vazios

df_focos_queimadas.loc[df_focos_queimadas.diasemchuva.isna(), 'diasemchuva'] = 0
df_focos_queimadas.loc[df_focos_queimadas.precipitacao.isna(), 'precipitacao'] = 0
df_focos_queimadas.loc[df_focos_queimadas.riscofogo.isna(), 'riscofogo'] = 0

In [11]:
# conferindo

df_focos_queimadas.isna().sum()

datahora             0
estado               0
municipio            0
bioma                0
diasemchuva          0
precipitacao         0
riscofogo            0
latitude             0
longitude            0
frp             611136
dtype: int64

**Converter nome dos estados em UF**

Para isso, foi criado um arquivo json, chave-valor, e o objetivo é substituir os nomes dos estados pelas suas UFs, para facilitar a visualização das informações no gráfico, quando houver necessidade de mostrá-los.

In [12]:
with open('dados/estados_map.json', encoding='utf-8') as json_file:
    estados_map = json.load(json_file)

estados_map

{'ACRE': 'AC',
 'ALAGOAS': 'AL',
 'AMAPA': 'AP',
 'AMAZONAS': 'AM',
 'BAHIA': 'BA',
 'CEARA': 'CE',
 'DISTRITO FEDERAL': 'DF',
 'ESPIRITO SANTO': 'ES',
 'GOIAS': 'GO',
 'MARANHAO': 'MA',
 'MATO GROSSO': 'MT',
 'MATO GROSSO DO SUL': 'MS',
 'MINAS GERAIS': 'MG',
 'PARA': 'PA',
 'PARAIBA': 'PB',
 'PARANA': 'PR',
 'PERNAMBUCO': 'PE',
 'PIAUI': 'PI',
 'RIO DE JANEIRO': 'RJ',
 'RIO GRANDE DO NORTE': 'RN',
 'RIO GRANDE DO SUL': 'RS',
 'RONDONIA': 'RO',
 'RORAIMA': 'RR',
 'SERGIPE': 'SE',
 'SANTA CATARINA': 'SC',
 'SAO PAULO': 'SP',
 'TOCANTINS': 'TO'}

In [13]:
df_focos_queimadas = df_focos_queimadas.replace({'estado': estados_map})

**Trabalhando com as datas**

Primeiro, se sabe que o fuso horário dos satélites é o GMT (UTC). Como a análise será feita numa região específica do globo, no caso o Brasil, pode-se converter o fuso do campo datahora para o de Brasília (-3 horas em relação ao GMT) que é o considerado o oficial brasileiro.

In [14]:
# Converter a datahora para um objeto data

df_focos_queimadas['datahora'] = pd.to_datetime(df_focos_queimadas['datahora'], errors='coerce')

# Converter o objeto datahora para o fuso de SP (que é igual o de Brasilia)

df_focos_queimadas['datahora_tz'] = df_focos_queimadas.apply(Funcoes.converter_fuso, axis=1)

In [15]:
# Converter a datahora com fuso de Brasília, para um objeto data

df_focos_queimadas['datahora_tz'] = pd.to_datetime(df_focos_queimadas['datahora_tz'], errors='coerce')

Depois será necessário separar data e hora, para facilitar trabalhar com ambos.

In [16]:
# Separar a data e e hora em colunas diferentes

df_focos_queimadas['data'] = Funcoes.converter_data_hora(df_focos_queimadas, 'datahora_tz', 'data')
df_focos_queimadas['hora'] = Funcoes.converter_data_hora(df_focos_queimadas, 'datahora_tz', 'hora')

In [17]:
df_focos_queimadas['dia_semana'] = df_focos_queimadas['datahora_tz'].dt.weekday # Dia da semana
df_focos_queimadas['mes'] = df_focos_queimadas['datahora_tz'].dt.month # Mês do ano

In [18]:
df_focos_queimadas.head(5)

,datahora,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp,datahora_tz,data,hora,dia_semana,mes
0,2015-11-07 16:30:00,PA,PORTEL,Amazonia,4.0,0.1,0.6,-2.245,-50.667,NaN,2015-11-07 14:30:00,2015-11-07,14:30:00,5,11
1,2015-04-02 16:46:00,TO,PEIXE,Cerrado,1.0,0.1,0.3,-11.994,-48.548,NaN,2015-04-02 13:46:00,2015-04-02,13:46:00,3,4
2,2015-03-23 17:47:00,MT,FELIZ NATAL,Amazonia,3.0,0.4,0.2,-12.185,-54.405,NaN,2015-03-23 14:47:00,2015-03-23,14:47:00,0,3
3,2015-02-21 17:39:00,RR,IRACEMA,Amazonia,20.0,0.0,1.0,2.247,-61.527,NaN,2015-02-21 15:39:00,2015-02-21,15:39:00,5,2
4,2015-08-02 17:25:00,PA,ORIXIMINA,Amazonia,0.0,0.1,0.8,-1.754,-55.686,NaN,2015-08-02 14:25:00,2015-08-02,14:25:00,6,8


**Risco fogo**

Vamos adicionar a coluna `riscofogo_nivel` para classificar os níveis do risco de fogo, que o próprio instituto utiliza: 

- Mínimo: abaixo de 0,15; 
- Baixo: de 0,15 a 0,4; 
- Médio: de 0,4 a 0,7; 
- Alto: de 0,7 a 0,95 ; 
- Crítico: acima de 0,95 até 1.

In [19]:
df_focos_queimadas['riscofogo_nivel'] = df_focos_queimadas.apply(Funcoes.classificar_risco_fogo, axis=1)

Vamos adicionar a coluna `riscofogo_categoria` para fazer uma classificação binária

- Abaixo de 0,7: 0 (Mínimo, Baixo, Médio)
- Acima de 0,7: 1 (Alto, Crítico)


In [20]:
df_focos_queimadas['riscofogo_categoria'] = df_focos_queimadas.apply(Funcoes.categorizar_risco_fogo, axis=1)

---

**Tratamento de dados para os modelos de machine learning**

Será criada uma cópia do dataframe anterior para preparar os dados para os modelos de machine learning. O dataframe original, será utilizado para a exploração de dados.

In [21]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [22]:
df_focos_queimadas_ml = df_focos_queimadas.copy()

In [23]:
# Excluir colunas que não serão utilizadas

df_focos_queimadas_ml.drop(['datahora', 'datahora_tz', 'data', 'hora', 'latitude', 'longitude'], axis=1, inplace=True)

In [24]:
labelencoder = LabelEncoder()

In [25]:
# Transformar os níveis de fogo em ids

df_focos_queimadas_ml['riscofogo_nivel_id'] = labelencoder.fit_transform(df_focos_queimadas_ml['riscofogo_nivel'])

In [26]:
# Transformar os municipios em ids

df_focos_queimadas_ml['municipio_id'] = labelencoder.fit_transform(df_focos_queimadas_ml['municipio'])

Transformar as colunas a seguir em colunas dummy, isso impede que elas sejam lidas pelo algoritmo como ordinais.

In [27]:
_df1 = pd.get_dummies(df_focos_queimadas_ml['bioma'], prefix="bioma")
_df2 = pd.get_dummies(df_focos_queimadas_ml['estado'], prefix="estado")

In [28]:
df_focos_queimadas_ml = pd.concat([df_focos_queimadas_ml, _df1], axis=1)
df_focos_queimadas_ml = pd.concat([df_focos_queimadas_ml, _df2], axis=1)

In [29]:
# Excluir colunas não utilizadas

df_focos_queimadas_ml.drop(['riscofogo_nivel', 'municipio', 'bioma', 'estado'], axis=1, inplace=True)

In [30]:
df_focos_queimadas_ml.head()

,diasemchuva,precipitacao,riscofogo,frp,dia_semana,mes,riscofogo_categoria,riscofogo_nivel_id,municipio_id,bioma_Amazonia,...,estado_PR,estado_RJ,estado_RN,estado_RO,estado_RR,estado_RS,estado_SC,estado_SE,estado_SP,estado_TO
0,4.0,0.1,0.6,NaN,5,11,0,3,3660,1,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0.1,0.3,NaN,3,4,0,1,3458,0,...,0,0,0,0,0,0,0,0,0,1
2,3.0,0.4,0.2,NaN,0,3,0,1,1635,1,...,0,0,0,0,0,0,0,0,0,0
3,20.0,0.0,1.0,NaN,5,2,1,2,2091,1,...,0,0,0,0,1,0,0,0,0,0
4,0.0,0.1,0.8,NaN,6,8,1,0,3210,1,...,0,0,0,0,0,0,0,0,0,0
